In [1]:
from IPython.display import HTML, display
display(HTML("<style>.container { width:95% !important; }</style>"))

# A Study Into Swath Coverage

___

For this notebook our goal is to develop an initial study into the camera coverage from the satellites projected onto Earth. In order to understand this we will need to understand orbital period for the satellites. 

https://keisan.casio.com/exec/system/1224665242

Above is an online calculator for an orbital period. For now the main take-away is for our altitude of interest (400-600 km) the orbital period varies only slightly 

|Altitude|Orbital Period|Flight Velocity|
|--------|--------------|---------------|
|     400|     1hr 32min|     6,778.14km|
|--------|--------------|---------------|
|     500|     1hr 34min|     6,878.14km|
|--------|--------------|---------------|
|     600|     1hr 36min|     6,978.14km|

Note for this calculation, to calculate the orbital period we have 

\begin{equation*}
    P = 2\pi\sqrt{\frac{a^3}{\mu}}.
\end{equation*}

Where $a$ is the semi-major axis and $\mu = GM$ where $G$ is Earth's gravitational force and $M$ is its mass. The calculation to compute $a$ uses the orbital period, at least within the Wiki article on orbital period. i.e. $a = \sqrt[3]{\frac{GMP^2}{4\pi^2}}$

But Wiki provides something a little further that is useful, they say "When a very small body is in a circular orbit barely above the surface of a sphere of any radius and mean density ρ (in kg/m3), the above equation simplifies to

\begin{equation*}
    P = \sqrt{\frac{3\pi}{G\rho}}
\end{equation*}

Thus the orbital period in low orbit depends only on the density of the central body, regardless of its size."

What the website looks into is, first it calculates the orbital velocity and then uses that to calculate the orbital period. Their equation is only based off altitude which from what I input earlier justifies the simplicity of the problem (our satellites are 'close' enough to earth). They have 

\begin{equation*}
    v = \sqrt{\frac{398,600.5}{6,378.14 + h}}
\end{equation*}

where $h$ is the altitude and $v$ is measured in km/s.

Within \textbf{Satellite Communications} by Timothy Pratt we found the value $6,378.14$ is the Earth's mean radius. The numerator is based on  

Finally they can calculate the orbital period by 

\begin{equation*}
    P = 2\pi \frac{6,738.14 + h}{v}.
\end{equation*}

___

Now we move towards being able to estimate coverage of the Earth. The Earth's equator, we are assuming once we cover the equator we've sufficiently covered the rest of the Earth, is $40,075$ km. Below is code written by Ricky where justification can be found within the "Modelling Camera Coverage" jupyter notebook.

In [2]:
import math
import numpy as np

In [15]:
def coverage(theta, h):
    l = 2 * h * np.tan( np.pi * ( theta / 2 ) / 180 )
    print("Swath = ", l, "km")
#    cov = l**2
#    return cov

With this we can now determine varying swath sizes based on our altitude.

In [16]:
coverage(30.0, 400)

Swath =  214.35935394489815 km


In [17]:
coverage(30.0, 600)

Swath =  321.53903091734725 km


Ok. Moving on we will assume that an orbital period of a satellite will be 90 minutes. Just for simplicity at the beginning. Given this, we can calculate how many times the satellite orbits the Earth in a single day. Well we have 24 hours in a day and 1.5 hours to orbit the Earth, so we orbit the Earth exactly 16 times in one day and pass the equator 32 times in a day. The only issue with this is, we only want to count coverage on the sunny side of the Earth. On the dark side we will not be measuring any evapotranspiration so we assume 16 passes a day.

Thus since we orbit the Earth 16 times in 24 hours, and the Earth orbits once in 24 hours, we can measure how far the Earth has rotated in one satellite orbital period. i.e. for one satellite period the Earth has rotated $\frac{40,075}{16} = 2504.6875$ km. 

Let's do a quick study to check in on overlap through days.

In [22]:
swath = 321
days = 12
x = [[0,swath]]
for i in range (1,16*days):
    a = (i*2504.6875) % 40075
    b = (i*2504.6875+hieght) % 40075
    x.append([a,b])
    
x

[[0, 321],
 [2504.6875, 2825.6875],
 [5009.375, 5330.375],
 [7514.0625, 7835.0625],
 [10018.75, 10339.75],
 [12523.4375, 12844.4375],
 [15028.125, 15349.125],
 [17532.8125, 17853.8125],
 [20037.5, 20358.5],
 [22542.1875, 22863.1875],
 [25046.875, 25367.875],
 [27551.5625, 27872.5625],
 [30056.25, 30377.25],
 [32560.9375, 32881.9375],
 [35065.625, 35386.625],
 [37570.3125, 37891.3125],
 [0.0, 321.0],
 [2504.6875, 2825.6875],
 [5009.375, 5330.375],
 [7514.0625, 7835.0625],
 [10018.75, 10339.75],
 [12523.4375, 12844.4375],
 [15028.125, 15349.125],
 [17532.8125, 17853.8125],
 [20037.5, 20358.5],
 [22542.1875, 22863.1875],
 [25046.875, 25367.875],
 [27551.5625, 27872.5625],
 [30056.25, 30377.25],
 [32560.9375, 32881.9375],
 [35065.625, 35386.625],
 [37570.3125, 37891.3125],
 [0.0, 321.0],
 [2504.6875, 2825.6875],
 [5009.375, 5330.375],
 [7514.0625, 7835.0625],
 [10018.75, 10339.75],
 [12523.4375, 12844.4375],
 [15028.125, 15349.125],
 [17532.8125, 17853.8125],
 [20037.5, 20358.5],
 [22542.1

So for how we've simplified this, it turns out that each day we start at exactly the same location. Note that with the exact orbital period we would start to veer off some (I think, we will look into this later).

___

Thus in order to cover all of the equator, we need enough satellites to cover the distance the Earth travels in one orbital period. For an altitude of 400 km, we have $\frac{2504.6875}{214} = 11.70414$. Hence it would take all 12 satellites in order to cover the Earth. For an altitude of 600 km, we have $\frac{2504.6875}{321} = 7.80276$ and thus 8 satellites here. Which kind of matches the table within the powerpoint provided by David.

___

From here on we will try to replicate their table.

In [21]:
swath = coverage(30.0, 485)

Swath =  259.910716658189 km


They get 254 and we get 260 pretty much for an altitude of 485 km. Their table then specifies them having 3 satellites taking 4 days. Dividing the Earth's rotation in one orbit by 260 km for the swath we get the need for 10 satellites to orbit. This is JUST to cover that distance for coverage in 1 day. If (and assuming rightly so) the orbit does not overlap exactly 3 satellites covering Earth in 4 days seems right.

In [23]:
swath = coverage(30.0, 410)

Swath =  219.71833779352062 km


They have 6 satellites in 2 days. We have 12 satellites in 1 day. So again it makes sense for 6 in 2 days.